In [11]:
import numpy as np

def csv_to_npz():
    test_data = np.genfromtxt('test_Ice.csv' ,dtype=float, delimiter=',',skip_header=0)
    train_data = np.genfromtxt('train_Ice.csv' ,dtype=float, delimiter=',',skip_header=0)
    
    test_x = test_data[1:, 1:-1]
    train_x = train_data[1:, 1:-1]

    test_y = test_data[1:, -1]
    train_y = train_data[1:, -1]

    np.savez_compressed('test.npz', X=test_x, Y=test_y)
    np.savez_compressed('train.npz', X=train_x, Y=train_y)

def npz_check():
    data = np.load('train.npz')
    print(data['X'].shape)
    return data

csv_to_npz()
npz_check()


(800, 2)


In [7]:
import random
import torch
import numpy as np
import os
from copy import deepcopy
import datetime
from tqdm.notebook import tqdm
from torch.utils.tensorboard import SummaryWriter
import params

from train_utils import *

model_type = 'simple_fc'  # 'att_context'
id = 'simple_fc'

# Set hyperparams:
missing = False
num_epochs = params.NUM_EPOCHS
batch_size = params.BATCH_SIZE
anneal_factor = params.ANNEALING_FACTOR
patience = params.PATIENCE

seed = 0

torch.cuda.synchronize() # comment
torch.cuda.empty_cache()

# set random seeds
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

# Other imports now
from data_utils import SalesDataset
from torch.utils.data import DataLoader

TRAIN = 'train.npz'
TEST = 'test.npz'

train_data = SalesDataset(TRAIN, randomize = False)
test_data = SalesDataset(TEST, randomize = False)

train_loader = DataLoader(train_data, batch_size, shuffle = True)
test_loader = DataLoader(test_data, batch_size, shuffle = True)
val_loader = test_loader

#------------------#
# Model Definition #
#------------------#

from hyperband import random_nn_params
from model import DeepVendorSimple
from loss_function import EuclideanLoss, CostFunction

best_overall_model = None
best_overall_cost = 100000.
best_model_idx = 0


for model_idx in tqdm(range(params.NUM_MODELS)):

    hidden_layers, n_nodes, lr, weight_decay = random_nn_params()

    model = DeepVendorSimple(n_hidden = hidden_layers, n_nodes = n_nodes)

    torch.cuda.empty_cache()
    #torch.cuda.memory_summary()
    #device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    device = torch.device('cpu')
    print(device)
    model = model.to(device)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=lr,
        weight_decay=weight_decay)

    criterion = EuclideanLoss(params.SHORTAGE_COST, params.HOLDING_COST)
    test_criterion = CostFunction(params.SHORTAGE_COST, params.HOLDING_COST)

    writer_path = os.path.join(id+'_idx_'+str(model_idx))
    writer = SummaryWriter(writer_path)

    best_model = None
    best_val_loss = 100000.0

    try:
        early_stopping = EarlyStopping(patience=patience, verbose=False)

        for epoch in tqdm(range(num_epochs)):
            torch.cuda.empty_cache()

            # Train model
            loss = discriminative_trainer(
                model=model,
                data_loader=train_loader,
                optimizer=optimizer,
                criterion=criterion)
            print(f'epoch: {epoch}, loss: {loss}')

            # log in tensorboard
            writer.add_scalar('Training/Prediction Loss', loss, epoch)

            # Eval model
            loss = discriminative_evaluate(model, val_loader, test_criterion)
            writer.add_scalar('Validation/Prediction Cost', loss, epoch)

            if loss < best_val_loss:
                best_val_loss = loss
                best_model = deepcopy(model)

            # Anneal LR
            early_stopping(loss, model)
            if early_stopping.early_stop:  # and epoch > params.MIN_EPOCH:
                print("Early stopping")
                break

    except KeyboardInterrupt:
        print('Stopping training. Now testing')

    # Test the model
    model = best_model
    torch.cuda.empty_cache()
    loss= discriminative_evaluate(model, test_loader, test_criterion)
    print('Test Prediction Cost: ', loss)

    if loss < best_overall_cost:
        best_overall_model = deepcopy(model)
        best_overall_cost = loss
        best_model_idx = model_idx

torch.save(model.state_dict(), os.path.join('best_model.pth'))


  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 368.92it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 1086.22it/s]

Generating network with 4 layers, each with [2 4 4 1] nodes
(2, array([2, 4, 4, 1]), 0.003748321662847925, 0.003481410351917467)
Generating network with 5 layers, each with [2 4 5 5 1] nodes
cpu
epoch: 0, loss: 20.47552085876465




  4%|███▎                                                                               | 1/25 [00:00<00:04,  5.62it/s]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 254.48it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 482.81it/s]

  8%|██████▋                                                                            | 2/25 [00:00<00:04,  5.08it/s]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

 42%|██████████████████████████████████                                               | 21/50 [00:00<00:00, 206.43it/s]

epoch: 1, loss: 20.0801735496521


100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 249.50it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 1448.50it/s]

 12%|█████████▉                                                                         | 3/25 [00:00<00:04,  4.83it/s]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 378.15it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 685.67it/s]

...

epoch: 9, loss: 20.1581418800354
EarlyStopping counter: 7 out of 10





100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 718.81it/s]

 44%|████████████████████████████████████                                              | 11/25 [00:02<00:03,  4.47it/s]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 355.55it/s]


100%|█████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 931.13it/s]

 48%|███████████████████████████████████████▎                                          | 12/25 [00:02<00:02,  4.85it/s]

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

epoch: 10, loss: 20.398601665496827
EarlyStopping counter: 8 out of 10
epoch: 11, loss: 20.271930084228515
EarlyStopping counter: 9 out of 10




100%|█████████████████████████████████████████████████████████████████████████████████| 50/50 [00:00<00:00, 339.39it/s]


 48%|███████████████████████████████████████▎                                          | 12/25 [00:03<00:03,  3.94it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:52<00:00,  5.22s/it]

epoch: 12, loss: 20.25641025543213
EarlyStopping counter: 10 out of 10
Early stopping
Test Prediction Cost:  54.70273993565486


In [8]:
best_model_idx

6